In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_continuousdr as file

In [2]:
# Args setup
ntrial = 50
n = 100
n_int = 100
front_door = False

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="16GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              front_door = front_door,
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-d191e9514835f74d056d7a5e921e553e>,
 <Future: pending, key: main-461ad8fcf53601b76c8904e3f792e7ce>,
 <Future: pending, key: main-7580b3e980e9a7b65c774529b140fa1c>,
 <Future: pending, key: main-8cc3342d3b8ee7988ac288c383d19e6e>,
 <Future: pending, key: main-bb6743a2ab7e98df9938c3d647ae24bb>,
 <Future: pending, key: main-67542f5beb72cd04ea0cb883054dbc2b>,
 <Future: pending, key: main-717e34581460525c19430cdb653ebd83>,
 <Future: pending, key: main-17adf27ebb9b069151c21eb0a246936d>,
 <Future: pending, key: main-d559c871793239969f7f79a8bd0f6682>,
 <Future: pending, key: main-a089a7cd7fd26bb230a95acb2873b7e3>,
 <Future: pending, key: main-624ed8b95940bd454e34c23650812081>,
 <Future: pending, key: main-805e49e53baccdaec346bf03068d3894>,
 <Future: pending, key: main-237c17e5525ff71d242aaa150e9383cb>,
 <Future: pending, key: main-78c586434ab3fad2bfe502183e8064e2>,
 <Future: pending, key: main-06cf2f4c64e85880c4f1e49dae1e1258>,
 <Future: pending, key: main-1cbbdafee0a

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "simulation_continuousdr_ntrial={0}_n={1}_frontdoor={2}.pt".format(ntrial,n, front_door)
          )